In [1]:
print("hello")

hello


In [2]:
from azureml.core import Workspace

# Connect to your workspace
ws = Workspace.from_config()  # Ensure you have a config file
print("Workspace connected:", ws.name)


Workspace connected: mlworkspace


In [3]:
from azureml.core import Dataset

# Load dataset
dataset = Dataset.get_by_name(ws, name='TermDepositDataset')  # Replace with your dataset name
data = dataset.to_pandas_dataframe()
print(data.head())

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
   age  balance  day  duration  campaign  previous  job_blue-collar  \
0   59     2343    5      1042         1         0            False   
1   56       45    5      1467         1         0            False   
2   41     1270    5      1389         1         0            False   
3   55     2476    5       579         1         0            False   
4   54      184    5       673         2         0            False   

   job_entrepreneur  job_housemaid  job_management  ...  month_may  month_nov  \
0             False          False           False  ...       True      False   
1             False          False           False  ...       True      False   
2             False          False           False  ...       True      False   
3             False          False           False  ...       True      False   
4

In [4]:
# Check for missing values
print("Missing values per column:")
print(data.isnull().sum())

# Verify column types
print("Column data types:")
print(data.dtypes)


Missing values per column:
age                    0
balance                0
day                    0
duration               0
campaign               0
previous               0
job_blue-collar        0
job_entrepreneur       0
job_housemaid          0
job_management         0
job_retired            0
job_self-employed      0
job_services           0
job_student            0
job_technician         0
job_unemployed         0
job_unknown            0
marital_married        0
marital_single         0
education_secondary    0
education_tertiary     0
education_unknown      0
contact_telephone      0
contact_unknown        0
month_aug              0
month_dec              0
month_feb              0
month_jan              0
month_jul              0
month_jun              0
month_mar              0
month_may              0
month_nov              0
month_oct              0
month_sep              0
poutcome_other         0
poutcome_success       0
poutcome_unknown       0
housing_new            

In [5]:
print(data.info())
print(data.describe(include='all'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11152 entries, 0 to 11151
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   age                  11152 non-null  int64
 1   balance              11152 non-null  int64
 2   day                  11152 non-null  int64
 3   duration             11152 non-null  int64
 4   campaign             11152 non-null  int64
 5   previous             11152 non-null  int64
 6   job_blue-collar      11152 non-null  bool 
 7   job_entrepreneur     11152 non-null  bool 
 8   job_housemaid        11152 non-null  bool 
 9   job_management       11152 non-null  bool 
 10  job_retired          11152 non-null  bool 
 11  job_self-employed    11152 non-null  bool 
 12  job_services         11152 non-null  bool 
 13  job_student          11152 non-null  bool 
 14  job_technician       11152 non-null  bool 
 15  job_unemployed       11152 non-null  bool 
 16  job_unknown          1

In [7]:
# Convert TabularDataset to DataFrame
data = dataset.to_pandas_dataframe()

# Separate features and target variable
X = data.drop(columns=['deposit_new'])  # Replace 'deposit_new' with your target column name
y = data['deposit_new']

# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)



{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
Training data shape: (8921, 40)
Testing data shape: (2231, 40)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)


Model Accuracy: 0.8520842671447781


In [9]:
import joblib
from azureml.core import Workspace, Model

# Save the model locally
joblib.dump(model, 'random_forest_model.pkl')
print("Model saved locally as 'random_forest_model.pkl'.")

# Connect to Azure ML workspace
ws = Workspace.from_config()

# Register the model
registered_model = Model.register(
    workspace=ws,
    model_name='random_forest_model',
    model_path='random_forest_model.pkl'  # Path to the saved model file
)
print(f"Model registered in Azure ML with name: {registered_model.name}")


Model saved locally as 'random_forest_model.pkl'.
Registering model random_forest_model
Model registered in Azure ML with name: random_forest_model


In [10]:
import joblib
import json
import os
import pandas as pd

def init():
    global model, expected_columns
    # Load the model from the environment
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], 'random_forest_model.pkl')
    model = joblib.load(model_path)

    # Expected columns based on training
    expected_columns = [
        'age', 'balance', 'day', 'duration', 'campaign', 'previous',
        'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management',
        'job_retired', 'job_self-employed', 'job_services', 'job_student',
        'job_technician', 'job_unemployed', 'job_unknown', 'marital_married',
        'marital_single', 'education_secondary', 'education_tertiary',
        'education_unknown', 'contact_telephone', 'contact_unknown',
        'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
        'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
        'month_sep', 'poutcome_other', 'poutcome_success', 'poutcome_unknown',
        'housing_new', 'loan_new'
    ]

def run(raw_data):
    try:
        # Parse input JSON
        input_data = json.loads(raw_data)

        # Ensure "data" key is present
        if "data" not in input_data:
            return {"error": "Invalid input format. 'data' key is missing."}

        # Convert input to DataFrame
        input_df = pd.DataFrame(input_data["data"])

        # Reorder and align columns
        input_df = input_df.reindex(columns=expected_columns, fill_value=0)

        # Make predictions
        predictions = model.predict(input_df)

        # Return predictions
        return {"predictions": predictions.tolist()}

    except Exception as e:
        return {"error": str(e)}


In [11]:
def run(raw_data):
    try:
        # Parse the input JSON
        input_data = json.loads(raw_data)

        # Debug: Log raw input
        print("Raw input data:", input_data)

        # Check if "data" key is present
        if "data" not in input_data:
            return {"error": "Invalid input format. 'data' key is missing."}

        # Convert input to DataFrame
        input_df = pd.DataFrame(input_data["data"])

        # Debug: Log DataFrame structure
        print("DataFrame structure after parsing input:")
        print(input_df.head())

        # Ensure all expected columns are present
        for col in expected_columns:
            if col not in input_df.columns:
                input_df[col] = 0 if input_df.get(col).dtype != 'bool' else False

        # Reorder the DataFrame to match training columns
        input_df = input_df[expected_columns]

        # Debug: Log DataFrame structure before prediction
        print("DataFrame structure before prediction:")
        print(input_df.head())

        # Predict using the model
        predictions = model.predict(input_df)

        # Debug: Log predictions
        print("Predictions:", predictions)

        return {"predictions": predictions.tolist()}

    except Exception as e:
        # Log error
        return {"error": str(e)}


In [12]:
import json
import pandas as pd
import joblib

# Load the trained model
model = joblib.load("random_forest_model.pkl")  # Update path as necessary

# Define expected columns
expected_columns = [
    'age', 'balance', 'day', 'duration', 'campaign', 'previous',
    'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management',
    'job_retired', 'job_self-employed', 'job_services', 'job_student',
    'job_technician', 'job_unemployed', 'job_unknown', 'marital_married',
    'marital_single', 'education_secondary', 'education_tertiary',
    'education_unknown', 'contact_telephone', 'contact_unknown',
    'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
    'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
    'month_sep', 'poutcome_other', 'poutcome_success', 'poutcome_unknown',
    'housing_new', 'loan_new'
]

# Simulate raw input
raw_data = json.dumps({
    "data": [
        {
            "age": 35,
            "balance": 2000,
            "day": 15,
            "duration": 300,
            "campaign": 2,
            "previous": 1,
            "job_blue-collar": False,
            "job_entrepreneur": False,
            "job_housemaid": False,
            "job_management": True,
            "job_retired": False,
            "job_self-employed": False,
            "job_services": False,
            "job_student": False,
            "job_technician": False,
            "job_unemployed": False,
            "job_unknown": False,
            "marital_married": True,
            "marital_single": False,
            "education_secondary": True,
            "education_tertiary": False,
            "education_unknown": False,
            "contact_telephone": True,
            "contact_unknown": False,
            "month_aug": False,
            "month_dec": False,
            "month_feb": False,
            "month_jan": True,
            "month_jul": False,
            "month_jun": False,
            "month_mar": False,
            "month_may": True,
            "month_nov": False,
            "month_oct": False,
            "month_sep": False,
            "poutcome_other": False,
            "poutcome_success": True,
            "poutcome_unknown": False,
            "housing_new": 1,
            "loan_new": 0
        }
    ]
})

# Simulate the `run()` function
def run(raw_data):
    try:
        # Parse input JSON
        input_data = json.loads(raw_data)
        print("Raw input data:", input_data)

        # Check for "data" key
        if "data" not in input_data:
            return {"error": "Invalid input format. 'data' key is missing."}

        # Convert to DataFrame
        input_df = pd.DataFrame(input_data["data"])
        print("DataFrame after parsing input:")
        print(input_df.head())

        # Add missing columns and reorder
        for col in expected_columns:
            if col not in input_df.columns:
                input_df[col] = 0 if input_df.get(col).dtype != 'bool' else False

        input_df = input_df[expected_columns]
        print("DataFrame before prediction:")
        print(input_df.head())

        # Predict
        predictions = model.predict(input_df)
        print("Predictions:", predictions)
        return {"predictions": predictions.tolist()}

    except Exception as e:
        return {"error": str(e)}

# Run the simulation
response = run(raw_data)
print("Response:", response)


Raw input data: {'data': [{'age': 35, 'balance': 2000, 'day': 15, 'duration': 300, 'campaign': 2, 'previous': 1, 'job_blue-collar': False, 'job_entrepreneur': False, 'job_housemaid': False, 'job_management': True, 'job_retired': False, 'job_self-employed': False, 'job_services': False, 'job_student': False, 'job_technician': False, 'job_unemployed': False, 'job_unknown': False, 'marital_married': True, 'marital_single': False, 'education_secondary': True, 'education_tertiary': False, 'education_unknown': False, 'contact_telephone': True, 'contact_unknown': False, 'month_aug': False, 'month_dec': False, 'month_feb': False, 'month_jan': True, 'month_jul': False, 'month_jun': False, 'month_mar': False, 'month_may': True, 'month_nov': False, 'month_oct': False, 'month_sep': False, 'poutcome_other': False, 'poutcome_success': True, 'poutcome_unknown': False, 'housing_new': 1, 'loan_new': 0}]}
DataFrame after parsing input:
   age  balance  day  duration  campaign  previous  job_blue-collar 

In [13]:
from azureml.core.environment import Environment

env = Environment(name="deployment_env")
env.python.conda_dependencies.add_pip_package("scikit-learn")
env.python.conda_dependencies.add_pip_package("pandas")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("azureml-defaults")


In [16]:
import os
print("Current working directory:", os.getcwd())


Current working directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith


In [17]:
import os
print("Does score.py exist in the current directory?", os.path.exists("score.py"))


Does score.py exist in the current directory? False


In [18]:
import os
print("Absolute path to score.py:", os.path.abspath("score.py"))


Absolute path to score.py: /mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith/score.py


In [20]:
import os
print("Current working directory:", os.getcwd())


Current working directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith


In [22]:
import os

def find_file(file_name, search_path):
    for root, dirs, files in os.walk(search_path):
        if file_name in files:
            return os.path.join(root, file_name)
    return None

# Search for score.py in your working directory
search_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters"  # Adjust the path if needed
file_name = "score.py"
file_path = find_file(file_name, search_path)

if file_path:
    print(f"score.py found at: {file_path}")
else:
    print("score.py not found!")


score.py not found!


In [23]:
import json
import joblib
import numpy as np
import pandas as pd

# Initialize the model
def init():
    global model
    # Load the model from the local file
    model_path = "random_forest_model.pkl"  # Ensure this is the name of your model file
    model = joblib.load(model_path)

# Handle scoring requests
def run(raw_data):
    try:
        # Parse the input data
        input_data = json.loads(raw_data)

        # Convert input data to DataFrame
        input_df = pd.DataFrame(input_data["data"])

        # Make predictions
        predictions = model.predict(input_df)

        # Return the predictions as a list
        return {"predictions": predictions.tolist()}
    except Exception as e:
        return {"error": str(e)}


In [24]:
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig, Model
from azureml.core import Workspace

# Load the workspace
ws = Workspace.from_config()

# Load the registered model
model = Model(ws, name="random_forest_model")  # Replace with your registered model's name

# Define the environment
env = Environment(name="deployment_env")
env.python.conda_dependencies.add_pip_package("scikit-learn")
env.python.conda_dependencies.add_pip_package("pandas")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("azureml-defaults")

# Set the inference configuration
inference_config = InferenceConfig(
    entry_script="/mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith/score.py",
    environment=env
)

# Define the deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model
service = Model.deploy(
    workspace=ws,
    name="random-forest-service",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)
print("Service deployed at:", service.scoring_uri)


/tmp/ipykernel_3469/2194677010.py:29: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(
Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 65732bbe-7959-483f-b0a3-f6c9234c7cea
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'r

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-01-10 11:44:21+00:00 Creating Container Registry if not exists..
2025-01-10 11:54:22+00:00 Registering the environment..
2025-01-10 11:54:26+00:00 Building image..
2025-01-10 12:02:55+00:00 Generating deployment configuration..
2025-01-10 12:02:57+00:00 Submitting deployment to compute..
2025-01-10 12:03:06+00:00 Checking the status of deployment random-forest-service..
2025-01-10 12:04:26+00:00 Checking the status of inference endpoint random-forest-service.
Failed


WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 65732bbe-7959-483f-b0a3-f6c9234c7cea
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'random_forest_model.pkl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'random_forest_model.pkl', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 65732bbe-7959-483f-b0a3-f6c9234c7cea\nMore in

In [39]:
import json
import joblib
import os
import pandas as pd

def init():
    global model
    # Path to the model file
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "random_forest_model.pkl")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        # Parse input data
        input_data = json.loads(raw_data)
        input_df = pd.DataFrame(input_data["data"])

        # Make predictions
        predictions = model.predict(input_df)

        return {"predictions": predictions.tolist()}
    except Exception as e:
        return {"error": str(e)}


In [42]:
from azureml.core import Model

# List all registered models in the workspace
models = Model.list(ws)
if not models:
    print("No models found in the workspace. Register the model first.")
else:
    for m in models:
        print(f"Name: {m.name}, Version: {m.version}")




Name: random_forest_model, Version: 1


In [43]:
import os
print("Model exists in current directory:", os.path.exists("random_forest_model.pkl"))


Model exists in current directory: True


In [44]:
# Get model path after registration
model = Model(ws, name="random_forest_model")
model_path = model.download(target_dir=".", exist_ok=True)
print("Model path:", model_path)


Model path: random_forest_model.pkl


In [45]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Create environment
env = Environment(name="deployment_env")
env.python.conda_dependencies.add_pip_package("scikit-learn")
env.python.conda_dependencies.add_pip_package("pandas")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("azureml-defaults")

# Define inference configuration
inference_config = InferenceConfig(
    entry_script="score.py",  # Ensure this file is correctly created in the current directory
    environment=env
)


In [46]:
# Define deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)


In [47]:
from azureml.core.model import Model

# Deploy the model
service = Model.deploy(
    workspace=ws,
    name="random-forest-service",
    models=[Model(ws, name="random_forest_model")],
    inference_config=inference_config,
    deployment_config=deployment_config
)

# Wait for deployment to complete
service.wait_for_deployment(show_output=True)
print("Service deployed at:", service.scoring_uri)


/tmp/ipykernel_3469/2720144126.py:4: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(
Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 15743735-364a-4ae6-baab-e93642df4f7f
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring f

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-01-10 12:58:27+00:00 Creating Container Registry if not exists.
2025-01-10 12:58:30+00:00 Use the existing image.
2025-01-10 12:58:30+00:00 Generating deployment configuration.
2025-01-10 12:58:32+00:00 Submitting deployment to compute.
2025-01-10 12:58:36+00:00 Checking the status of deployment random-forest-service..
2025-01-10 12:59:42+00:00 Checking the status of inference endpoint random-forest-service.
Failed


WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 15743735-364a-4ae6-baab-e93642df4f7f
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: random-forest-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 36425093288646f28df895f52dfcdd46.azurecr.io/azureml/azureml_8b7968530d5aaeec584b6c1f8c78d268 locally. P

In [25]:
ls /mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith


classification.ipynb*         random_forest_model.pkl*
classification.ipynb.amltmp*  score.py*


In [26]:
print(service.get_logs())


None


In [27]:
print(service.state)


Failed


In [28]:
import os

print("Score.py exists:", os.path.exists("/mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith/score.py"))
print("Model exists:", os.path.exists("/mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith/random_forest_model.pkl"))


Score.py exists: True
Model exists: True


In [29]:
logs = service.get_logs()
if logs:
    print(logs)
else:
    print("No logs available. Check deployment configuration and files.")


No logs available. Check deployment configuration and files.


In [30]:
import os
print("Score.py exists:", os.path.exists("/mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith/score.py"))
print("Model exists:", os.path.exists("/mnt/batch/tasks/shared/LS_root/mounts/clusters/adekiitn4faith1/code/Users/adekiitn4faith/random_forest_model.pkl"))



Score.py exists: True
Model exists: True


In [33]:
from azureml.core import Model

# List all models in the workspace
models = Model.list(ws)
if models:
    for m in models:
        print(f"Name: {m.name}, Version: {m.version}")
else:
    print("No models found in the workspace.")




Name: random_forest_model, Version: 1


In [34]:
print("Service scoring URI:", service.scoring_uri)


Service scoring URI: None


In [38]:
from azureml.core import Model

# List all models in the workspace
models = Model.list(ws)
if not models:
    print("No models registered in the workspace.")
else:
    for m in models:
        print(f"Name: {m.name}, Version: {m.version}, Path: {m.id}")




Name: random_forest_model, Version: 1, Path: random_forest_model:1
